In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_database/weather_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Aklavik,CA,2022-04-10 13:43:47,68.2191,-135.0107,-17.12,98,85,2.33,overcast clouds
1,1,Rikitea,PF,2022-04-10 13:39:28,-23.1203,-134.9692,78.62,73,1,15.68,clear sky
2,2,Presque Isle,US,2022-04-10 13:43:47,46.6811,-68.0159,41.18,100,100,0.00,mist
3,3,Jamestown,US,2022-04-10 13:43:47,42.0970,-79.2353,35.19,52,100,14.97,overcast clouds
4,4,Manaus,BR,2022-04-10 13:43:48,-3.1019,-60.0250,82.89,78,40,2.30,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 75
What is your desired maximum temperature for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,2022-04-10 13:39:28,-23.1203,-134.9692,78.62,73,1,15.68,clear sky
4,4,Manaus,BR,2022-04-10 13:43:48,-3.1019,-60.0250,82.89,78,40,2.30,scattered clouds
5,5,Vaini,TO,2022-04-10 13:41:10,-21.2000,-175.2000,77.16,94,40,5.75,scattered clouds
14,14,Hermanus,ZA,2022-04-10 13:43:28,-34.4187,19.2345,81.81,41,0,12.57,clear sky
16,16,Bambous Virieux,MU,2022-04-10 13:43:51,-20.3428,57.7575,80.85,74,40,11.50,light intensity shower rain
21,21,Anuradhapura,LK,2022-04-10 13:43:52,8.3565,80.4173,76.30,94,100,4.88,light rain
30,30,Sao Joao Da Barra,BR,2022-04-10 13:43:54,-21.6403,-41.0511,80.69,76,36,6.31,scattered clouds
33,33,Wana,PK,2022-04-10 13:43:55,32.2989,69.5725,81.27,9,13,5.50,few clouds
36,36,Katima Mulilo,NaN,2022-04-10 13:43:56,-17.5000,24.2667,75.07,69,100,7.43,light rain
39,39,Hithadhoo,MV,2022-04-10 13:43:25,-0.6000,73.0833,84.24,72,25,9.04,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,2022-04-10 13:39:28,-23.1203,-134.9692,78.62,73,1,15.68,clear sky
4,4,Manaus,BR,2022-04-10 13:43:48,-3.1019,-60.0250,82.89,78,40,2.30,scattered clouds
5,5,Vaini,TO,2022-04-10 13:41:10,-21.2000,-175.2000,77.16,94,40,5.75,scattered clouds
14,14,Hermanus,ZA,2022-04-10 13:43:28,-34.4187,19.2345,81.81,41,0,12.57,clear sky
16,16,Bambous Virieux,MU,2022-04-10 13:43:51,-20.3428,57.7575,80.85,74,40,11.50,light intensity shower rain
...,...,...,...,...,...,...,...,...,...,...,...
665,665,Santa Rosa,PH,2022-04-10 13:47:04,14.3122,121.1114,82.38,78,100,7.45,overcast clouds
667,667,Cockburn Town,TC,2022-04-10 13:47:04,21.4612,-71.1419,78.58,77,57,13.60,broken clouds
669,669,Petatlan,MX,2022-04-10 13:47:05,17.5167,-101.2667,80.64,48,12,0.47,few clouds
671,671,Isangel,VU,2022-04-10 13:47:05,-19.5500,169.2667,80.06,82,21,8.75,few clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = " "
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,78.62,clear sky,-23.1203,-134.9692,
4,Manaus,BR,82.89,scattered clouds,-3.1019,-60.0250,
5,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,
14,Hermanus,ZA,81.81,clear sky,-34.4187,19.2345,
16,Bambous Virieux,MU,80.85,light intensity shower rain,-20.3428,57.7575,
21,Anuradhapura,LK,76.30,light rain,8.3565,80.4173,
30,Sao Joao Da Barra,BR,80.69,scattered clouds,-21.6403,-41.0511,
33,Wana,PK,81.27,few clouds,32.2989,69.5725,
39,Hithadhoo,MV,84.24,scattered clouds,-0.6000,73.0833,
43,Puerto Ayora,EC,78.78,overcast clouds,-0.7393,-90.3518,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.") 

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [ ]:
# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "vacation_search/weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig